# Setup

In [ ]:
%pip install magtrack matplotlib

In [ ]:
import magtrack
from magtrack.simulation import simulate_beads
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

## Simulation
To start we will simulate a stack of 100 images of beads. In each frame the bead will drift slightly to the right.

In [ ]:
n = 100
roi = 64
nm_per_px = 100.0

xy_offset = roi*nm_per_px/2
x_true = np.linspace(-200, 200, n)
y_true = np.zeros_like(x_true)
z_true = np.zeros_like(x_true)
xyz_true = np.stack([x_true, y_true, z_true], axis=1)
stack = simulate_beads(xyz_true, size_px=roi, nm_per_px=nm_per_px)
x_true += xy_offset
y_true += xy_offset
first_image = stack[:, :, 0]

plt.figure(figsize=(9, 3))

plt.subplot(1, 4, 1)
plt.imshow(first_image, cmap='gray')
plt.title('First Frame')

plt.subplot(1, 4, 2)
plt.plot(x_true)
plt.title('X Position')
plt.ylabel('x (nm)')
plt.xlabel('frame #')

plt.subplot(1, 4, 3)
plt.plot(y_true)
plt.title('Y Position')
plt.ylabel('y (nm)')
plt.xlabel('frame #')

plt.subplot(1, 4, 4)
plt.plot(z_true)
plt.title('Z Position')
plt.ylabel('z (nm)')
plt.xlabel('frame #')

plt.tight_layout()
plt.show()

# Example 1: Get XY with Center-of-Mass with CPU
We will now try getting a rough xy position with the center_of_mass algorithm

In [ ]:
x_fit, y_fit = magtrack.center_of_mass(stack, background="median")
x_fit *= nm_per_px
y_fit *= nm_per_px

print(f'x: {x_fit}, y: {y_fit}')

Now we will plot those out

In [ ]:
plt.figure(figsize=(6, 9))

plt.subplot(3, 2, 1)
plt.imshow(first_image, cmap='gray')
plt.plot(x_true[0]/nm_per_px, y_true[0]/nm_per_px, 'b+', label='True')
plt.plot(x_fit[0]/nm_per_px, x_fit[0]/nm_per_px, 'rx', label='Fit')
plt.legend()
plt.title('First Frame')

plt.subplot(3, 2, 2)
plt.imshow(first_image, cmap='gray')
plt.plot(x_true[0]/nm_per_px, y_true[0]/nm_per_px, 'b+', label='True')
plt.plot(x_fit[0]/nm_per_px, x_fit[0]/nm_per_px, 'rx', label='Fit')
plt.xlim(x_true[0]/nm_per_px-5, x_true[0]/nm_per_px+5)
plt.ylim(y_true[0]/nm_per_px-5, y_true[0]/nm_per_px+5)
plt.legend()
plt.title('First Frame (Zoom)')

plt.subplot(3, 2, 3)
plt.plot(x_true, label='True')
plt.plot(x_fit, label='Fit')
plt.legend()
plt.title('X Position')
plt.ylabel('x (nm)')
plt.xlabel('frame #')
plt.ylim(x_true.min()-20, x_true.max()+20)

plt.subplot(3, 2, 4)
plt.plot(y_true, label='True')
plt.plot(y_fit, label='Fit')
plt.legend()
plt.title('Y Position')
plt.ylabel('y (nm)')
plt.xlabel('frame #')
plt.ylim(y_true.min()-20, y_true.max()+20)

plt.subplot(3, 2, 5)
plt.plot(abs(x_true-x_fit)/x_true*100)
plt.title('X Percent Error')
plt.ylabel('x error (%)')
plt.xlabel('frame #')
plt.ylim(0, 2)

plt.subplot(3, 2, 6)
plt.plot(abs(y_true-y_fit)/y_true*100)
plt.title('Y Percent Error')
plt.ylabel('y error (%)')
plt.xlabel('frame #')
plt.ylim(0, 2)

plt.tight_layout()
plt.show()

# Example 2: Refine the XY using sub-pixel auto convolution on the GPU
You can run any magtrack function on the CPU or GPU. The GPU is often much faster. If you are using Google Colab you will need to change the GPU by going to Runtime → Change runtime type → GPU. If you get stuck, just move on to the next example which uses the CPU.

In [ ]:
# Verify a GPU exists
try:
  cp.cuda.runtime.getDeviceCount()
except Exception:
  print('Error! '*10 + '\n')
  print("No GPU runtime.\n")
  print("In Colab: Runtime → Change runtime type → GPU. \n\n")
else:
  dev = cp.cuda.runtime.getDevice()
  props = cp.cuda.runtime.getDeviceProperties(dev)
  name = props["name"].decode() if isinstance(props["name"], (bytes, bytearray)) else props["name"]
  print(f"GPU: {name} | CC {props['major']}.{props['minor']} | {props['totalGlobalMem']/1e9:.1f} GB")

In [ ]:
# Move the stack to the GPU's RAM
stack_gpu = cp.asarray(stack)

# Estimate the XY position with the center-of-mass
x_com_gpu, y_com_gpu = magtrack.center_of_mass(stack_gpu)
x_com = cp.asnumpy(x_com_gpu) * nm_per_px # Move back to regular RAM
y_com = cp.asnumpy(y_com_gpu) * nm_per_px

# Get a better estimate with the sub-pixel auto-convolution
x_ac_gpu, y_ac_gpu = magtrack.auto_conv_para_fit(stack_gpu, x_com_gpu, y_com_gpu)
x_ac = cp.asnumpy(x_ac_gpu) * nm_per_px
y_ac = cp.asnumpy(y_ac_gpu) * nm_per_px

# Finally, we can measure the average percent error with each method
x_err = float(np.mean(np.abs(x_com - x_true) / x_true * 100))
y_err = float(np.mean(np.abs(y_com - y_true) / y_true * 100))
print(f'Center-of-mass error:    x: {x_err:.3f}%, y: {y_err:.3f}%')
x_err = float(np.mean(np.abs(x_ac - x_true) / x_true * 100))
y_err = float(np.mean(np.abs(y_ac - y_true) / y_true * 100))
print(f'Auto-convolution error:  x: {x_err:.3f}%, y: {y_err:.3f}%')

# Example 3: Refine XY on the CPU with Quadratic Interpolation
This example keeps everything on the CPU so that it works on any computer. We start with the center-of-mass positions
and then call `magtrack.qi_refine_xy` to get smoother, sub-pixel estimates. The steps below show how to
prepare the inputs, run the function, and read the results.

1. **Reuse the simulated stack** from the setup cells above.
2. **Get starting positions** with `magtrack.center_of_mass`.
3. **Refine the coordinates** by passing the stack and starting points into `qi_refine_xy`.
4. **Convert the refined values to nanometers** (nm) so they match the `x_true` and `y_true` arrays.
5. **Compare the average percent error** to see the improvement over the basic center-of-mass result.

Feel free to run this cell multiple times—the commands are safe to rerun and do not change anything else in the notebook.


In [ ]:
# Step 1 & 2: get starting positions in pixel units
x_com_px, y_com_px = magtrack.center_of_mass(stack, background="median")

# Step 3: refine with quadratic interpolation on the CPU
x_qi_px, y_qi_px = magtrack.qi_refine_xy(stack, x_com_px, y_com_px)
x_qi_px, y_qi_px = magtrack.qi_refine_xy(stack, x_qi_px, y_qi_px)
x_qi_px, y_qi_px = magtrack.qi_refine_xy(stack, x_qi_px, y_qi_px)

# Step 4: convert everything to nanometers for an apples-to-apples comparison
x_com_nm = x_com_px * nm_per_px
y_com_nm = y_com_px * nm_per_px
x_qi_nm = x_qi_px * nm_per_px
y_qi_nm = y_qi_px * nm_per_px

# Step 5: check how much accuracy we gained
def percent_error(true_values, measured_values):
    true_values = np.asarray(true_values, dtype=float)
    measured_values = np.asarray(measured_values, dtype=float)
    denominator = np.maximum(np.abs(true_values), 1e-9)
    return float(np.mean(np.abs(true_values - measured_values) / denominator) * 100.0)

x_err_com = percent_error(x_true, x_com_nm)
y_err_com = percent_error(y_true, y_com_nm)
x_err_qi = percent_error(x_true, x_qi_nm)
y_err_qi = percent_error(y_true, y_qi_nm)

print(f"Center-of-mass average error:        x = {x_err_com:.3f}% | y = {y_err_com:.3f}%")
print(f"Quadratic interpolation average error: x = {x_err_qi:.3f}% | y = {y_err_qi:.3f}%")

# Bonus: plot the refined positions so you can see the smoother trace
plt.figure(figsize=(8, 4))
plt.plot(x_true, label='True X (nm)', color='black', linewidth=2)
plt.plot(x_com_nm, label='Center-of-mass (nm)', linestyle='--')
plt.plot(x_qi_nm, label='Quadratic interp (nm)', linestyle=':')
plt.xlabel('Frame number')
plt.ylabel('X position (nm)')
plt.title('Refining the X trace with qi_refine_xy')
plt.legend()
plt.tight_layout()
plt.show()


# Example 4: Build a Z-LUT for axial fitting
This final example shows how to turn simulated bead images into the lookup table
that `magtrack.lookup_z` expects. We will create a stack that sweeps a bead
through a range of z-positions, convert each frame into a radial profile, and pack
those profiles into a Z-LUT (Z look-up table).


In [ ]:
# Simulate a reference stack that scans the bead in z
z_reference = np.arange(-5000, 5100, 100)  # nanometers
xyz_reference = np.column_stack([
    np.zeros_like(z_reference),
    np.zeros_like(z_reference),
    z_reference,
])
reference_stack = simulate_beads(xyz_reference, size_px=roi, nm_per_px=nm_per_px)

# Convert each frame into a radial profile centered on the bead
center_px = np.full(z_reference.shape, roi / 2, dtype=float)
reference_profiles = magtrack.radial_profile(
    reference_stack, center_px, center_px
)

# Assemble the Z-LUT: the first row stores the z positions,
# the remaining rows contain the radial profiles
zlut = np.vstack([z_reference, reference_profiles])
print(f'Z-LUT shape: {zlut.shape}')

In [ ]:
plt.figure(figsize=(6, 4))
plt.imshow(reference_profiles.T, cmap='gray', vmin=0, vmax=1, aspect=0.75)
plt.ylabel('Reference z (nm)')
plt.xlabel('Radius (pixels)')
plt.title('Z-LUT')
y_ticks = np.linspace(0, reference_profiles.shape[1]-1, 5).astype(int)
plt.yticks(y_ticks)
plt.gca().set_yticklabels(z_reference[y_ticks])
plt.tight_layout()
plt.show()


With the Z-LUT prepared, we can analyze a fresh stack, extract its radial profiles,
and estimate the bead's axial position. The lookup function returns a sub-frame
interpolation of the best matching reference profiles.


In [ ]:
# Create a new stack with a different z trajectory
z_true_eval = np.linspace(-3000, 3000, 100) + 1000.0 * np.sin(np.linspace(0, 20 * np.pi, 100))
xyz_eval = np.column_stack([
    np.zeros_like(z_true_eval),
    np.zeros_like(z_true_eval),
    z_true_eval,
])
eval_stack = simulate_beads(xyz_eval, size_px=roi, nm_per_px=nm_per_px)

eval_profiles = magtrack.radial_profile(
    eval_stack, np.full(z_true_eval.shape, roi / 2, dtype=float),
    np.full(z_true_eval.shape, roi / 2, dtype=float),
)
z_fit = magtrack.lookup_z(eval_profiles, zlut)

plt.figure(figsize=(6, 6))

plt.subplot(2, 1, 1)
plt.plot(z_true_eval, label='True', linewidth=2)
plt.plot(z_fit, label='Fit', linestyle='--')
plt.xlabel('Frame number')
plt.ylabel('Z (nm)')
plt.title('Z fit from lookup_z')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(z_fit - z_true_eval, 'r-', label='Residual')
plt.xlabel('Frame number')
plt.ylabel('Residual (nm)')
plt.legend()

plt.tight_layout()
plt.show()


# Example 5: Explore `radial_profile` step by step
The `magtrack.radial_profile` helper summarizes each image by averaging the light intensity at equal distances from a chosen center point. This is a friendly way to see whether your bead images are nicely circular.

Use the checklist below if you are new to Python—each item corresponds to a short code block right after this text.

1. **Pick a frame number** you want to study.
2. **Convert the bead position from nanometers to pixels.** The function expects pixel units.
3. **Call `magtrack.radial_profile`.** The output is one brightness value for each radius bin.
4. **Plot the image and its profile side-by-side** to build an intuition for what you are seeing.

Feel free to rerun this cell with different frame numbers. Nothing else in the notebook will be changed.

---


In [ ]:
# Step 1: pick a frame number you want to inspect
frame_to_inspect = 0  # try changing this number and rerun the cell

# Step 2: convert the bead center from nanometers to pixel units
x_center_px = np.array([x_true[frame_to_inspect] / nm_per_px], dtype=float)
y_center_px = np.array([y_true[frame_to_inspect] / nm_per_px], dtype=float)
print(f'Center (pixels): x={x_center_px[0]:.2f}, y={y_center_px[0]:.2f}')

# Step 3: slice the stack so the shape matches (pixels, pixels, frames)
single_frame_stack = stack[:, :, frame_to_inspect:frame_to_inspect + 1]

# Step 4: call radial_profile and unpack the 2D output into a simple 1D curve
single_profile = magtrack.radial_profile(single_frame_stack, x_center_px, y_center_px)
single_profile = single_profile[:, 0]

# Step 5: build a radius axis (in nanometers) for the plot
radii_pixels = np.arange(single_profile.size, dtype=float)
radii_nm = radii_pixels * nm_per_px

# Display the selected frame next to its radial profile
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(single_frame_stack[:, :, 0], cmap='gray')
plt.scatter(x_center_px, y_center_px, color='red', label='Chosen center')
plt.title(f'Frame {frame_to_inspect}')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.legend(loc='lower left')

plt.subplot(1, 2, 2)
plt.plot(radii_nm, single_profile, marker='o')
plt.title('Radial intensity profile')
plt.xlabel('Radius (nm)')
plt.ylabel('Average intensity')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Bonus: run radial_profile on the full stack to understand the output shape
all_profiles = magtrack.radial_profile(stack, x_true / nm_per_px, y_true / nm_per_px)
print(f'The full radial_profile output has shape {all_profiles.shape} (radius bins × frames)')


# Example 6: Check bead sharpness with `fft_profile`
The `magtrack.fft_profile` helper looks at the same bead images in the frequency domain. It radial-averages the 2D Fourier transform so you can spot whether the bead contains the high-frequency detail you expect. Follow the notes below if you are new to notebooks:

1. **Copy the frame before calling `fft_profile`.** The helper multiplies the stack by a Gaussian window; copying keeps the original images untouched.
2. **Decide how much oversampling you want.** The default `oversample=4` gives smooth curves without taking too long.
3. **Plot the results two ways.** The left panel shows the log-scaled 2D FFT, while the right panel shows the radial average in cycles per pixel.
4. **Optionally run on the whole stack.** The printout at the end shows the array shape when you pass every frame at once.


In [ ]:
# Step 1: make a safe copy of the frame so fft_profile's weighting does not alter `stack`
fft_frame = stack[:, :, frame_to_inspect:frame_to_inspect + 1].astype(float).copy()

# Step 2: choose the oversampling factor for smoother frequency bins
fft_oversample = 4
fft_profile_result = magtrack.fft_profile(
    fft_frame,
    x_center_px,
    y_center_px,
    oversample=fft_oversample,
    rmin=0.0,
    rmax=1.0,
)
fft_profile_1d = fft_profile_result[:, 0]

# Step 3: build a frequency axis (cycles per pixel)
freq_bins = np.arange(fft_profile_1d.size, dtype=float) / fft_oversample
freq_cycles_per_px = freq_bins / fft_frame.shape[0]

if fft_profile_1d.max() > 0:
    normalized_profile = fft_profile_1d / fft_profile_1d.max()
else:
    normalized_profile = fft_profile_1d

# Step 4: view the 2D FFT alongside the radial average
fft_image = np.fft.fftshift(np.fft.fft2(single_frame_stack[:, :, 0]))
fft_magnitude = np.log1p(np.abs(fft_image))

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.imshow(fft_magnitude, cmap='magma')
plt.title('Log-scaled FFT magnitude')
plt.xlabel('Frequency pixel (x)')
plt.ylabel('Frequency pixel (y)')
plt.colorbar(label='log(1 + amplitude)', fraction=0.046, pad=0.04)

plt.subplot(1, 2, 2)
plt.plot(freq_cycles_per_px, normalized_profile, marker='o')
plt.title('Radial FFT profile')
plt.xlabel('Frequency (cycles per pixel)')
plt.ylabel('Normalized amplitude')
plt.xlim(0, 0.5)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Bonus: process the entire stack (use .copy() because fft_profile modifies its input)
full_fft_profiles = magtrack.fft_profile(
    stack.copy(),
    x_true / nm_per_px,
    y_true / nm_per_px,
    oversample=fft_oversample,
    rmax=1.0,
)
print(f'The fft_profile output has shape {full_fft_profiles.shape} (frequency bins × frames)')
